In [10]:
import sys
import numpy as np
import pandas as pd
import os 
import gc
import math
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import BaggingRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [11]:
path  = 'D:/MyDocument/Project/CarSalesPrediction/'
# train_sales  = pd.read_csv(path+'train_sales_data.csv')
# train_search = pd.read_csv(path+'train_search_data.csv')
# train_user   = pd.read_csv(path+'train_user_reply_data.csv')
# evaluation_public = pd.read_csv(path+'evaluation_public.csv')
# submit_example    = pd.read_csv(path+'submit_example.csv')
# data = pd.concat([train_sales, evaluation_public], ignore_index=True)
# data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
# data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth'])
# data['label'] = data['salesVolume']
# data['id'] = data['id'].fillna(0).astype(int)
train_sales  = pd.read_csv(path+'train_sales_data.csv')
train_search = pd.read_csv(path+'train_search_data.csv')
train_user_reply = pd.read_csv(path+'train_user_reply_data.csv')
evaluation_public = pd.read_csv(path+'evaluation_public.csv')
# submit_example    = pd.read_csv(path+'submit_example.csv')
data = pd.concat([train_sales, evaluation_public], ignore_index=True)
data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user_reply, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])

for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
data['mt'] = (data['regYear']-2016)*12+data['regMonth']

tmp = data['salesVolume'].groupby([data['adcode'], data['regYear'], data['regMonth']]).sum().reset_index()
tmp.rename(columns={'salesVolume':'adcode_year_month_sales_all'}, inplace=True)
data = data.merge(tmp, 'left', on=['adcode', 'regYear', 'regMonth'])
data['adcode_year_month_sales_all'] = data['adcode_year_month_sales_all']-data['salesVolume']

tmp = data['salesVolume'].groupby([data['adcode'], data['regYear'], data['regMonth'], data['bodyType']]).sum().reset_index()
tmp.rename(columns={'salesVolume':'adcode_year_month_bodyType_sales_all'}, inplace=True)
data = data.merge(tmp, 'left', on=['adcode', 'regYear', 'regMonth', 'bodyType'])
data['adcode_year_month_bodyType_sales_all'] = data['adcode_year_month_bodyType_sales_all']-data['salesVolume']

tmp = data['salesVolume'].groupby([data['model'],data['regYear'], data['regMonth']]).sum().reset_index()
tmp.rename(columns={'salesVolume':'model_year_month_sales_all'}, inplace=True)
data = data.merge(tmp, 'left', on=['model', 'regYear', 'regMonth'])
data['model_year_month_sales_all'] = data['model_year_month_sales_all']-data['salesVolume']

In [5]:
# data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])

In [6]:
# #LabelEncoder
# for i in ['bodyType', 'model']:
#     data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
# data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']

In [17]:
# tmp = data['salesVolume'].groupby([data['adcode'], data['regYear'], data['regMonth']]).sum().reset_index()
# tmp.rename(columns={'salesVolume':'adcode_year_month_sales_all'}, inplace=True)
# data = data.merge(tmp, 'left', on=['adcode', 'regYear', 'regMonth'])

# tmp = data['salesVolume'].groupby([data['adcode'], data['regYear'], data['regMonth'], data['bodyType']]).sum().reset_index()
# tmp.rename(columns={'salesVolume':'adcode_year_month_bodyType_sales_all'}, inplace=True)
# data = data.merge(tmp, 'left', on=['adcode', 'regYear', 'regMonth', 'bodyType'])

# tmp = data['salesVolume'].groupby([data['model'],data['regYear'], data['regMonth']]).sum().reset_index()
# tmp.rename(columns={'salesVolume':'model_year_month_sales_all'}, inplace=True)
# data = data.merge(tmp, 'left', on=['model', 'regYear', 'regMonth'])

In [12]:
data.loc[(data['regMonth']==1) & (data['regYear']==2016) & (data['model']==1) & (data['adcode'] != 110000)][['salesVolume','adcode']]

salesVolume  adcode
22        389.0  310000
23        306.0  530000
24        260.0  150000
26        757.0  510000
27        275.0  340000
28        890.0  370000
29        387.0  140000
30       8330.0  440000
31        801.0  450000
32       1685.0  320000
33        393.0  360000
34        788.0  130000
35        850.0  410000
36       1438.0  330000
37        762.0  420000
38        678.0  430000
39       1115.0  350000
40       1304.0  210000
41        188.0  500000
42        240.0  610000
43        467.0  230000

In [13]:
data.loc[0]['province']

'上海'

In [14]:
def get_stat_feature(df_):   
    df = df_.copy()
    stat_feat = []
    df['model_adcode'] = df['adcode'] + df['model']
    df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
    for col in tqdm(['salesVolume','popularity']):
        # shift
        for i in [1,2,3,4,5,6]:
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
            df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
#     for col in tqdm(['label']):
#         for i in [0,1,2,3]:
#             stat_feat.append('left_model_adcode_mt_{}_{}'.format(col,i))
#             df['model_adcode_mt_{}_{}'.format(col,-i)] = df['model_adcode_mt'] - i
#             df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,-i))
#             df['left_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
    
#     for col in tqdm(['label','popularity']):
#         # shift
#         for pair in [(1,2), (1,3), (2,3), (2,4), (3,4), (3,5), (1,6)]:
#             stat_feat.append('rise_model_adcode_mt_{}_{}_{}'.format(col,pair[0],pair[1]))
#             df['rise_model_adcode_mt_{}_{}_{}'.format(col,pair[0],pair[1])] = df['shift_model_adcode_mt_{}_{}'.format(col,pair[0])] - df['shift_model_adcode_mt_{}_{}'.format(col,pair[1])]
#     for col in tqdm(['label','popularity']):
#         # shift
#         stat_feat.append('mean_model_adcode_mt_{}'.format(col))
#         df['mean_model_adcode_mt_{}'.format(col)] = 0
#         for i in [1,2,3,4,5,6]:
#             df['mean_model_adcode_mt_{}'.format(col)] += 1.0/math.pow(2,i)*df['shift_model_adcode_mt_{}_{}'.format(col,i)]
    #         df['mean_model_adcode_mt_{}'.format(col)] = df['shift_model_adcode_mt_{}_{}'.format(col,pair[0])] - df['shift_model_adcode_mt_{}_{}'.format(col,pair[1])]
    return df,stat_feat

In [15]:
data_df, stat_feat = get_stat_feature(data)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.37it/s]


In [16]:
def score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
#     data['label'] = np.expm1(data['label'].values)
#     data['pred_label'] = np.expm1(data['pred_label'].values)
#     data['label'] = np.expm1(data['label'].values)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

In [17]:
def get_model_type(train_x,train_y,valid_x,valid_y,m_type='lgb'):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
                                max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
                                n_estimators=2000, subsample=0.9, colsample_bytree=0.7,
                                )
#         model = BaggingRegressor(model, n_estimators=8, random_state=0, n_jobs=1, max_samples=0.9,)
#         model.fit(train_x, train_y)
        
        model.fit(train_x, train_y, 
                      eval_set=[(train_x, train_y),(valid_x, valid_y)], 
                      categorical_feature=cate_feat, 
                      early_stopping_rounds=100, verbose=100)
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.05, n_estimators=2000, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)   
    return model

In [18]:
def get_train_model(df_, m, m_type='lgb'):
    df = df_.copy()
    # 数据集划分
    st = 7
    all_idx   = (df['mt'].between(st , m-1))
    train_idx = (df['mt'].between(st , m-5))
    valid_idx = (df['mt'].between(m-4, m-4))
    test_idx  = (df['mt'].between(m  , m  ))
    print('all_idx  :',st ,m-1)
    print('train_idx:',st ,m-5)
    print('valid_idx:',m-4,m-4)
    print('test_idx :',m  ,m  )  
    # 最终确认
    train_x = df[train_idx][features]
    train_y = df[train_idx]['label']
    valid_x = df[valid_idx][features]
    valid_y = df[valid_idx]['label']   
    # get model
    model = get_model_type(train_x,train_y,valid_x,valid_y,m_type)  
    # offline
    df['pred_label'] = model.predict(df[features])
#     df['pred_label'] = np.expm1(df['pred_label'].values)
    best_score = score(df[valid_idx]) 
    # online
    if m_type == 'lgb':
        model.n_estimators = model.best_iteration_ + 100
        model.fit(df[all_idx][features], df[all_idx]['label'])
    elif m_type == 'xgb':
        model.n_estimators = model.best_iteration + 100
        model.fit(df[all_idx][features], df[all_idx]['label'])
    df['forecastVolum'] = model.predict(df[features]) 
#     df['forecastVolum'] = np.expm1(df['forecastVolum'].values)
    print('valid mean:',df[valid_idx]['pred_label'].mean())
    print('true  mean:',df[valid_idx]['label'].mean())
    print('test  mean:',df[test_idx]['forecastVolum'].mean())
    # 阶段结果
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)  
    return sub,df[valid_idx]['pred_label']

In [20]:
for month in [25,26,27,28]: 
    m_type = 'xgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = stat_feat
    cate_feat = ['adcode','bodyType','model','regMonth']+['regYear']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
# sub['forecastVolum'] = np.expm1(sub['forecastVolum'].values)
sub[['id','forecastVolum']].round().astype(int).to_csv('xgb_sales.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 23.21it/s]


17 17
all_idx  : 7 24
train_idx: 7 20
valid_idx: 21 21
test_idx : 25 25
[0]	validation_0-rmse:978.278	validation_1-rmse:1046.8
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:936.784	validation_1-rmse:1004.68
[200]	validation_0-rmse:330.465	validation_1-rmse:353.543
[300]	validation_0-rmse:173.042	validation_1-rmse:174.022
[400]	validation_0-rmse:157.021	validation_1-rmse:161.938
[500]	validation_0-rmse:147.468	validation_1-rmse:159.927
[600]	validation_0-rmse:140.022	validation_1-rmse:159.187
[700]	validation_0-rmse:131.992	validation_1-rmse:159.048
Stopping. Best iteration:
[663]	validation_0-rmse:135.04	validation_1-rmse:158.127

0.796248363745279
valid mean: 657.4486694335938
true  mean: 649.3121212121212
test  mean: 527.0776977539062


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 22.03it/s]


17 17
all_idx  : 7 25
train_idx: 7 21
valid_idx: 22 22
test_idx : 26 26
[0]	validation_0-rmse:982.996	validation_1-rmse:1007.37
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:941.434	validation_1-rmse:965.709
[200]	validation_0-rmse:330.705	validation_1-rmse:378.42
[300]	validation_0-rmse:170.345	validation_1-rmse:267.044
Stopping. Best iteration:
[283]	validation_0-rmse:177.077	validation_1-rmse:264.395

0.6928207727854294
valid mean: 516.0877075195312
true  mean: 616.5537878787878
test  mean: 324.3305969238281


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 22.38it/s]


17 17
all_idx  : 7 26
train_idx: 7 22
valid_idx: 23 23
test_idx : 27 27
[0]	validation_0-rmse:984.537	validation_1-rmse:1071.89
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:942.968	validation_1-rmse:1028.99
[200]	validation_0-rmse:331.882	validation_1-rmse:352.249
[300]	validation_0-rmse:174.252	validation_1-rmse:158.604
[400]	validation_0-rmse:155.866	validation_1-rmse:150.774
[500]	validation_0-rmse:147.736	validation_1-rmse:147.49
[600]	validation_0-rmse:140.345	validation_1-rmse:147.921
Stopping. Best iteration:
[506]	validation_0-rmse:146.892	validation_1-rmse:147.256

0.8103832910811859
valid mean: 663.9490356445312
true  mean: 673.0143939393939
test  mean: 354.7147521972656


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 23.54it/s]


17 17
all_idx  : 7 27
train_idx: 7 23
valid_idx: 24 24
test_idx : 28 28
[0]	validation_0-rmse:989.888	validation_1-rmse:1451.3
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:948.225	validation_1-rmse:1408.39
[200]	validation_0-rmse:330.536	validation_1-rmse:657.385
[300]	validation_0-rmse:171.819	validation_1-rmse:401.73
[400]	validation_0-rmse:156.088	validation_1-rmse:386.26
[500]	validation_0-rmse:147.513	validation_1-rmse:380.234
[600]	validation_0-rmse:141.211	validation_1-rmse:375.887
[700]	validation_0-rmse:136.592	validation_1-rmse:376.835
Stopping. Best iteration:
[611]	validation_0-rmse:140.401	validation_1-rmse:375.334

0.6711077916765928
valid mean: 819.188232421875
true  mean: 899.8204545454546
test  mean: 310.9769287109375


In [30]:
# lgm_sales = pd.read_csv('E:/JupyterNotebook/lgb_sales.csv')
# xgb_sales = pd.read_csv('E:/JupyterNotebook/xgb_sales.csv')
# # lgm_sales1 = pd.read_csv('E:/JupyterNotebook/evaluation_public1.csv')
# # lgb_sales = pd.read_csv('E:/JupyterNotebook/submit.csv')
# # lgm_sales = pd.read_csv('E:/JupyterNotebook/lgm_evaluation_public.csv')
# # xgb_sales = pd.read_csv('E:/JupyterNotebook/xgb_sales.csv')
import pandas as pd
# rule_sales = pd.read_csv('E:/JupyterNotebook/rule_ccf_car_sales.csv')
xgb = pd.read_csv('D:/MyDocument/JupyterNotebook/xgb_sales.csv')
lgb = pd.read_csv('D:/MyDocument/JupyterNotebook/lgb_sales.csv')
lstm = pd.read_csv('D:/MyDocument/Project/CarSalesPrediction/salesvolumn_forecast/evaluation_public.csv')
df = lstm
df['forecastVolum'][:1320] = xgb['forecastVolum'][:1320]*0.7+lgb['forecastVolum'][:1320]*0.2+lstm['forecastVolum'][:1320]*0.1
df['forecastVolum'][1320:2640] = xgb['forecastVolum'][1320:2640]*0.66+lgb['forecastVolum'][1320:2640]*0.17+lstm['forecastVolum'][1320:2640]*0.17
df['forecastVolum'][2640:3960] = xgb['forecastVolum'][2640:3960]*0.65+lgb['forecastVolum'][2640:3960]*0.2+lstm['forecastVolum'][2640:3960]*0.15
df['forecastVolum'][3960:] = xgb['forecastVolum'][3960:]*0.65+lgb['forecastVolum'][3960:]*0.2+lstm['forecastVolum'][3960:]*0.15
# df['forecastVolum'][2640:] = 5*ml_sales['forecastVolum'][2640:]/6+rule_sales['forecastVolum'][2640:]/6
# df['forecastVolum'][1320:2640] = 3*ml_sales['forecastVolum'][1320:2640]/4+rule_sales['forecastVolum'][1320:2640]/4
# df['forecastVolum'] = lgm_sales['forecastVolum']/3 + xgb_sales['forecastVolum']/2 + rule_sales['forecastVolum']/6
# df['forecastVolum'] = 1*lgm_sales['forecastVolum']/3 + 2*xgb_sales['forecastVolum']/3
# df['forecastVolum'][2640:3960] = 5*df['forecastVolum'][2640:3960]/6+rule_sales['forecastVolum'][2640:3960]/6
# df['forecastVolum'][3960:] = 4*df['forecastVolum'][3960:]/5+rule_sales['forecastVolum'][3960:]/5
# df['forecastVolum'][1320:1320*3] = top['forecastVolum'][1320:1320*3]*0.85+lstm['forecastVolum'][1320:1320*3]*0.10+rule_sales['forecastVolum'][1320:1320*3]*0.05
# df['forecastVolum'][1320*3:] = top['forecastVolum'][1320*3:]*0.90+lstm['forecastVolum'][1320*3:]*0.05+rule_sales['forecastVolum'][1320*3:]*0.05
df[['id','forecastVolum']].round().astype(int).to_csv('evaluation_public.csv', index=False)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [31]:
import pandas as pd
top1 = pd.read_csv('D:/MyDocument/JupyterNotebook/evaluation_public_highest11.csv')
top2 = pd.read_csv('D:/MyDocument/JupyterNotebook/evaluation_public.csv')
top = top1
top['forecastVolum'] = (top2['forecastVolum'] - top1['forecastVolum'])/top1['forecastVolum']
print(top['forecastVolum'][0:1320].sum())
print(top['forecastVolum'][1320:2640].sum())
print(top['forecastVolum'][2640:3960].sum())
print(top['forecastVolum'][3960:].sum())
top.round(3).astype(float).to_csv('calculation.csv', index=False)

1.2835268612185433
6.004342029881973
0.4001965972262962
5.899528510749816
